In [4]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Define the path to the directory
directory_path = r'C:\Users\shaif\Downloads\Compressed\Invasive_Life_Stage'

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for folder in os.listdir(directory_path):
    #print(class_folder)
    clsp = os.path.join(directory_path,folder)  
    for class_folder in os.listdir(clsp):
        #print(class_folder)
        class_path = os.path.join(clsp, class_folder)    
        #print(class_path)
        for image_file in os.listdir(class_path):
            if image_file.endswith('.png'):
                image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
                img = Image.open(image_path).convert('RGB')
                img = img.resize((32, 32))
                img_array = np.array(img)
            
            # Append image and label to lists
                X_train.append(img_array)
                Y_train.append(class_label)
                #print(class_label)
    class_label = class_label + 1

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')


X_train shape: (49227, 32, 32, 3)
Y_train shape: (49227,)


In [5]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
Y_train_encoded = to_categorical(Y_train)
indices = np.arange(len(X_train))
np.random.shuffle(indices)
X_train_shuffled = X_train[indices]
Y_train_encoded_shuffled = Y_train_encoded[indices]
X_train, X_test, y_train, y_test = train_test_split(X_train_shuffled, Y_train_encoded_shuffled, test_size=0.2, random_state=42)

In [11]:
from tensorflow import keras
from keras.models import load_model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling1D

model_path = r"C:\Users\shaif\Downloads\invasive_ssl.h5"
model = load_model(model_path)
#model.layers[0].trainable = False
x = model.layers[-8].output  
output = Dense(5, activation='softmax')(x)
new_model = keras.models.Model(inputs=model.input, outputs=output)
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, mode='max', verbose=1)
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = new_model.fit(X_train, y_train, batch_size=32, epochs=100,validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
985/985 [==============================] - 63s 55ms/step - loss: 157.0787 - accuracy: 0.4077 - val_loss: 29.1292 - val_accuracy: 0.4146
Epoch 2/100
985/985 [==============================] - 50s 51ms/step - loss: 21.4489 - accuracy: 0.4315 - val_loss: 12.7934 - val_accuracy: 0.3812
Epoch 3/100
985/985 [==============================] - 54s 55ms/step - loss: 7.3389 - accuracy: 0.4578 - val_loss: 8.3072 - val_accuracy: 0.5475
Epoch 4/100
985/985 [==============================] - 55s 56ms/step - loss: 12.6336 - accuracy: 0.4612 - val_loss: 8.6090 - val_accuracy: 0.4286
Epoch 5/100
985/985 [==============================] - 64s 65ms/step - loss: 5.0935 - accuracy: 0.4775 - val_loss: 2.3090 - val_accuracy: 0.5732
Epoch 6/100
985/985 [==============================] - 76s 78ms/step - loss: 2.9578 - accuracy: 0.4925 - val_loss: 1.9954 - val_accuracy: 0.4822
Epoch 7/100
985/985 [==============================] - 69s 70ms/step - loss: 2.6602 - accuracy: 0.5003 - val_loss: 5.9289 - 

In [12]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
predictions = new_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
balanced_accuracy = balanced_accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Balanced Accuracy:", balanced_accuracy)
print("Test Accuracy:", accuracy)

Test Balanced Accuracy: 0.3002308718757795
Test Accuracy: 0.5384927889498273


In [ ]:
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = ResNet50(include_top=False, weights=None, input_shape=(32,32,3), pooling="avg")
x =  Dense(500)(base_model.output)
output = Dense(5, activation='softmax')(x)
ne_model = Model(inputs=base_model.input, outputs=output)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, mode='max', verbose=1)
ne_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = ne_model.fit(X_train, y_train, batch_size=32, epochs=100,validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
985/985 [==============================] - 59s 55ms/step - loss: 1.3516 - accuracy: 0.6551 - val_loss: 2.1893 - val_accuracy: 0.6652
Epoch 2/100
985/985 [==============================] - 55s 55ms/step - loss: 0.7126 - accuracy: 0.7301 - val_loss: 0.5955 - val_accuracy: 0.7899
Epoch 3/100
985/985 [==============================] - 59s 60ms/step - loss: 0.6249 - accuracy: 0.7663 - val_loss: 0.8270 - val_accuracy: 0.6770
Epoch 4/100
985/985 [==============================] - 60s 60ms/step - loss: 0.6286 - accuracy: 0.7662 - val_loss: 1.3351 - val_accuracy: 0.5413
Epoch 5/100
985/985 [==============================] - 54s 55ms/step - loss: 0.6261 - accuracy: 0.7673 - val_loss: 8.4699 - val_accuracy: 0.7075
Epoch 6/100
985/985 [==============================] - 51s 51ms/step - loss: 0.7215 - accuracy: 0.7354 - val_loss: 1.1967 - val_accuracy: 0.6633
Epoch 7/100
985/985 [==============================] - 58s 59ms/step - loss: 0.6004 - accuracy: 0.7748 - val_loss: 0.9703 - val_ac

In [28]:
from sklearn.metrics import accuracy_score
predictions = ne_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Accuracy:", accuracy)

Test Accuracy: 0.6181190331098924


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = MobileNetV2(include_top=False, weights=None, input_shape=(32,32,3), pooling="avg")
x =  Dense(500)(base_model.output)
output = Dense(5, activation='softmax')(x) 
new_model = Model(inputs=base_model.input, outputs=output)
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
new_model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

In [ ]:
from sklearn.metrics import accuracy_score
predictions = new_model.predict(np.array(X_test).astype('float32'))
accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))
print("Test Accuracy:", accuracy)